lightgbm (Light Gradient Boosting Machine)

- Microsoft Reserch (MSR) 를 중심으로 한 팀에서 발표

- xgboost와 매우 비슷 (속도가 더 빠르고 가볍다)

- XGB 등 이전의 decision tree 기반 boosting은 높은 정밀도를 기대하는 반면, 

- 분할하는데 시간이 걸림


특징

- level wise에서 leaf wise : 손실이 가장 줄어드는 노드에서 나누기

- histogram based : 연속값이 특징량을 히스토그램화 하여 bin 단위로 분할

- gradient-based one-side sampling (GOSS) : 기울기가 적은 데이터는 랜덤 샘플링

- Exclusive Feature Bundling (EFB) : 복수의 특징량을 bundle 하고 정리해 하나의 특징량과 같이 취급

In [1]:
# %pip uninstall lightgbm -y

In [10]:
%pip install lightgbm==3.3.5

Note: you may need to restart the kernel to use updated packages.


In [11]:
import lightgbm as lgb

부스팅 : 이전모델의 결과를 받아서 학습하는 것들
오차가 많으면 영향을 많이 받음

xg : 오차는 적당히 알아서 잘라버릴 수 있는 것

light : 좀 더 가볍고 빠르게 가능함

In [12]:
lgb.__version__

'3.3.5'

In [13]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
cancer = load_breast_cancer()

x = cancer.data
y = cancer.target

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [16]:
"""
boosting_type : default 'gdbt'
    - gdbt : Gradient Boosting Decision Tree
    - dart : Dropout meet Multiple Additiv Regression Trees
    - rf : Random Forest


Dropout : 가끔씩 몇 개는 사용하지 않겠다 (가중치가 없어짐 / 과대적합 문제를 해결)
"""

model = lgb.LGBMClassifier(boosting_type="gbdt",
                           max_depth=5,
                           n_jobs=-1,
                           random_state=1)

In [17]:
# early_stopping_rounds=10 
#학습하다가 학습이 잘 안될 때 
# 스코어가 크게 잘 안바뀔 때 10번 동안 잘 안바뀌면 학습 멈추겠다

eval_set = [(x_test, y_test)]
model.fit(x_train, y_train, eval_set=eval_set,
          early_stopping_rounds=10,
          verbose=True)

[1]	valid_0's binary_logloss: 0.589722
[2]	valid_0's binary_logloss: 0.538687
[3]	valid_0's binary_logloss: 0.494425
[4]	valid_0's binary_logloss: 0.458171
[5]	valid_0's binary_logloss: 0.430399
[6]	valid_0's binary_logloss: 0.404291
[7]	valid_0's binary_logloss: 0.375154
[8]	valid_0's binary_logloss: 0.350736
[9]	valid_0's binary_logloss: 0.332197
[10]	valid_0's binary_logloss: 0.310245
[11]	valid_0's binary_logloss: 0.296321
[12]	valid_0's binary_logloss: 0.280628
[13]	valid_0's binary_logloss: 0.26601
[14]	valid_0's binary_logloss: 0.253103
[15]	valid_0's binary_logloss: 0.238807
[16]	valid_0's binary_logloss: 0.232099
[17]	valid_0's binary_logloss: 0.219775
[18]	valid_0's binary_logloss: 0.211003
[19]	valid_0's binary_logloss: 0.204291
[20]	valid_0's binary_logloss: 0.195394
[21]	valid_0's binary_logloss: 0.189031
[22]	valid_0's binary_logloss: 0.183785
[23]	valid_0's binary_logloss: 0.178688
[24]	valid_0's binary_logloss: 0.173643
[25]	valid_0's binary_logloss: 0.168697
[26]	valid

c:\Users\heeji\anaconda3\envs\dsde04\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\heeji\anaconda3\envs\dsde04\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(max_depth=5, random_state=1)

In [18]:
# 58번째의 학습이 가장 잘 됨
model.best_iteration_

58

In [19]:
# 로스를 로그 함수로 돌린 평가 지표를 활용함
model.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('binary_logloss',
                           0.11892239895181878)])})